In [2]:
!pip install pulp

In [4]:
import pulp as op
import numpy as np

In [2]:
p = np.array([1,2,3]) #precios de las variedades.
#B = np.array([[0,0,0],[0,0,0]]) #matriz binaria de camas por variedad sembrada.
w = np.array([150,250,350,20,5,12,22,1000]) #capacidad de siembra de los bancos.
v = np.array([100,350,1]) # cantidades de variedades disponibles para sembrar.
n = v.shape[0] #Cantidad de variedades.
m = w.shape[0] #Cantidad de camas.
L=np.array([[0,0,0]]) #Restricciones de NO compatibilidad por variedad.

In [34]:
concat = lambda t,e : str(t)+str(e)
beds = [concat(xt,xe) for xt,xe in zip(np.repeat(['Bed'],m*n),np.repeat(np.arange(1,m+1),n))]
varieties = [concat(xt,xe) for xt,xe in zip(np.repeat(['_Var'],m*n),np.tile(np.arange(1,n+1),m))]
bedsXvars = [concat(xt,xe) for xt,xe in zip(beds,varieties)]
c=dict(zip(bedsXvars, np.tile(-p,m)))
A=dict(zip(bedsXvars,np.repeat(w,n)))

In [35]:
prob = op.LpProblem("Plannting_plan",op.LpMinimize)
beds=op.LpVariable.dicts("BedsVar",bedsXvars,lowBound=0,upBound=1,cat='Integer')
prob += op.lpSum([c[i]*beds[i] for i in bedsXvars]) # Función de Costo, Precio de la variedad * X
prob +=op.lpSum([A[i]*beds[i] for i in bedsXvars])>=v.sum()


In [ ]:

for i in np.arange(0,m): #Se crea un restricción por cama.
    p1 = np.zeros(i*n) #Cantidad de ceros adelante segun el indice de la cama
    p2 = np.ones(n) #Cantidad de unos que representan la restriccion
    p3 = np.zeros(((m*n)-((i*n)+n))) #cantidad de ceros al final
    r = dict(zip(bedsXvars,np.concatenate((p1,p2,p3))))

prob +=op.lpSum([A[i]*beds[i] for i in bedsXvars])>=v.sum()
    
Aeq[m:]=np.tile(L,m) #Restricciones de NO compatibilidad de flor


beq = np.zeros(m+L.shape[0])
beq[:m]=np.ones(m)
beq


In [37]:
prob.solve()
print("Status: ",op.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Status:  Optimal
BedsVar_Bed1_Var1 = 1.0
BedsVar_Bed1_Var2 = 1.0
BedsVar_Bed1_Var3 = 1.0
BedsVar_Bed2_Var1 = 1.0
BedsVar_Bed2_Var2 = 1.0
BedsVar_Bed2_Var3 = 1.0
BedsVar_Bed3_Var1 = 1.0
BedsVar_Bed3_Var2 = 1.0
BedsVar_Bed3_Var3 = 1.0
BedsVar_Bed4_Var1 = 1.0
BedsVar_Bed4_Var2 = 1.0
BedsVar_Bed4_Var3 = 1.0
BedsVar_Bed5_Var1 = 1.0
BedsVar_Bed5_Var2 = 1.0
BedsVar_Bed5_Var3 = 1.0
BedsVar_Bed6_Var1 = 1.0
BedsVar_Bed6_Var2 = 1.0
BedsVar_Bed6_Var3 = 1.0
BedsVar_Bed7_Var1 = 1.0
BedsVar_Bed7_Var2 = 1.0
BedsVar_Bed7_Var3 = 1.0
BedsVar_Bed8_Var1 = 1.0
BedsVar_Bed8_Var2 = 1.0
BedsVar_Bed8_Var3 = 1.0
